# Dataset description
* 32 participants [s01 to s32]
* 40 pieces of music
* 40 channels out of which the first 32 are relevant 
* 8064 readings recorded at 128Hz


In [2]:
from deap_pre_process import *

In [4]:
eeg1d,eeg2d,val,aro = get_subject_data(subject_no=1,media_no=1)

In [5]:
len(eeg2d.reshape(-1,9,9,1))

8064

In [27]:
eeg2d.shape

(8064, 9, 9)

# Making the dummy network


In [7]:
from rnn_function import *
from cnn_function import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [8]:
#Constant :
WINDOW_SIZE = 64
#For rnn layers
INPUT_NODES = 32
HIDDEN_ST = 32
HOLD_PROBA = 0.5
RNN_INPUT_SIZE = 32

#For cnn layers
CONV_IP_SHAPE = [-1,9,9,1]
CONV1_KERNEL = [4,4,1,32]
CONV2_KERNEL = [4,4,32,64]
CONV3_KERNEL = [4,4,64,128]
REDUCTION_KERNEL = [1,1,128*WINDOW_SIZE,13]

#For dnn layers

In [28]:
#Resetting tf default graph
tf.reset_default_graph()
#CNN placeholders and model variables
x_data_cnn = tf.placeholder(dtype = tf.float32,shape=(None,9,9),name='cnn_data')
x_image = tf.reshape(x_data_cnn,CONV_IP_SHAPE,name='cnn_image')

#LAYERS
conv1 = conv_layer(x_image,CONV1_KERNEL,name="convo_l1")
conv2 = conv_layer(conv1,CONV2_KERNEL,name="convo_l2")
conv3 = conv_layer(conv2,CONV3_KERNEL,name="convo_l3")
#Depth variable
depth = tf.reshape(conv3,[-1,9,9,128* WINDOW_SIZE])
#Size reduction conv layer
reduced_data = conv_layer(depth,REDUCTION_KERNEL,name="conv_reduced")


#RNN placeholder and model variables
x_data_rnn = tf.placeholder(dtype = tf.float32,shape=(None,WINDOW_SIZE,INPUT_NODES),name='rnn_data') #TIME_STEP = WINDOW_SIZE
keep_prob = tf.placeholder(dtype=tf.float32)

#Making cells
cell1 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cell2 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cells = [cell1,cell2]
final_cell = create_Stacked_cell(cells)

#Laying out the network
output, states = create_RNN(final_cell,x_data_rnn)
#Unstacking the output
output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')
#Selecting the final output form the layer
rnn_output = output[-1]

#Placeholder for true values
y_true = tf.placeholder(dtype=tf.float32,shape=(None,2)) #[Valance, Arousal]

In [33]:
#Flattening the layer
cnn_flattened = tf.reshape(reduced_data,shape=(1,9*9*13),name="cnn_flattened_layer")
rnn_flattened = tf.reshape(rnn_output,shape=(-1,32),name="flattened_layer")


#loss =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=concat,logits=y_true))

# REMAINING:
* Final dnn layer
* Loss and Optimizer 
* Some DNN stuff that was done to the LSTM input
* Session